In [2]:
import pandas as pd
import os
import json
from tqdm import tqdm

In [3]:
def extract_ref(ref:dict):
    info = ref["ref-info"]
    if "ref-title" in info.keys():
        if "ref-publicationyear" in info.keys():
            year = info["ref-publicationyear"]["@first"]
        else:
            year = "unknow"
        return year,info["ref-title"]["ref-titletext"]
    return (None,None)
def extract_pub(data:dict):
    abstracts = data["head"]["abstracts"]
    title = data["head"]["citation-title"]
    sourcetitle = data["head"]["source"]["sourcetitle"]
    return title,sourcetitle,abstracts

def extract_affiliation(data:dict):
    def get_affiname(affiliation:list[dict]|dict,col:str):
        if isinstance(affiliation,dict):
            return [affiliation[col] if col in affiliation.keys() else None]
        return [i[col] if col in i.keys() else None for i in affiliation] 
    
    data = data["abstracts-retrieval-response"]["affiliation"]
    return get_affiname(data,"affilname"), get_affiname(data,"affiliation-city"),get_affiname(data,"affiliation-country"),

def extract_json_file(file_name:str):
    list_ref = []
    
    with open(file_name,encoding="utf8") as f:
        try:
            data = json.load(f)
        except:
            print(file_name)
            return None,None
        affilname,affilcity,affilcountry = extract_affiliation(data)
        data = data["abstracts-retrieval-response"]["item"]["bibrecord"]
        title,sourcetitle,abstracts = extract_pub(data)
        new_pub = (title,sourcetitle,abstracts,affilname,affilcity,affilcountry)
        if data["tail"] is not None:
            list_referece = data["tail"]["bibliography"]["reference"]
            if isinstance(list_referece,dict):
                year_ref,title_ref = extract_ref(list_referece)
                if year_ref != None and title_ref != None:
                    list_ref.append((title,sourcetitle,year_ref,title_ref))
            else:
                for i in list_referece:
                    year_ref,title_ref = extract_ref(i)
                    if year_ref != None and title_ref != None:
                        list_ref.append((title,sourcetitle,year_ref,title_ref))
        return new_pub,list_ref

In [4]:
list_ref = []
list_pub = []

for pub_year in os.listdir(r"./Project"):
    for pub_name in tqdm(os.listdir(r"./Project/" + pub_year)):
        file_name = r"./Project/" + pub_year + "/"+ pub_name
        new_pub,list_single_ref = extract_json_file(file_name)
        list_ref.extend(list_single_ref)
        list_pub.append(new_pub)

  0%|          | 0/2792 [00:00<?, ?it/s]

100%|██████████| 2890/2890 [00:06<00:00, 472.15it/s]


In [66]:
df_ref_raw = pd.DataFrame(list_ref,columns=["title","sourcetitle","year","title_ref"])
df_pub_raw = pd.DataFrame(list_pub,columns=["title","sourcetitle","abstracts","affilname","affilcity","affilcountry"])
df_pub_raw["link"] = None

In [67]:
df_ref_raw.to_csv("ref_raW.csv",index=False)
df_pub_raw.to_csv("pub_raw.csv",index=False)

# Emb

In [87]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import ast
# Create TF-IDF representation

import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

def clean_text(text):
    
    # Remove text between parentheses
    text = re.sub(r'\([^)]*\)', ' ', text)
    
    # Remove punctuation
    text = re.sub(r'[^\w\s]', ' ', text)
    

    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Convert text to lowercase
    text = text.lower()
    
    # Tokenize text
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a single string
    cleaned_text = ' '.join(tokens)
    
    return cleaned_text

def proprocess_pub_raw(df_pub_raw:pd.DataFrame):

    df_pub_raw_explode = df_pub_raw.explode(["affilname","affilcity","affilcountry"])
    affiliate_df = df_pub_raw_explode[["affilname","affilcity","affilcountry"]].drop_duplicates().reset_index(drop=True).reset_index(names="id")
    df_pub_raw_explode = df_pub_raw_explode.merge(affiliate_df,how="left",on=["affilname","affilcity","affilcountry"])
    df_pub_merge_aff = df_pub_raw_explode.groupby("title")["id"].apply(list).to_frame().reset_index()
    df_pub_raw = df_pub_raw.merge(df_pub_merge_aff,on="title")[["title","abstracts","id","link"]].rename(columns={"id":"affiliation_id"})
    df_pub_raw = df_pub_raw.reset_index(names="id")
    
    return df_pub_raw,affiliate_df

def preprocess_df_ref(df_ref:pd.DataFrame,df_pub:pd.DataFrame):
    df_ref["title_ref"] = df_ref["title_ref"].str.lower()
    df_count = df_ref.groupby("title_ref").count()
    df_ref_new = df_ref[df_ref["title_ref"].isin(df_count[df_count["title"] > 1].index)]
    df_ref_new = df_ref_new.drop(columns=["sourcetitle"])
    df_merge = df_ref_new.merge(df_ref_new[["title_ref","title"]],on=["title_ref"],how="left",suffixes=('','_2'))
    df_merge = df_merge[df_merge["title"] != df_merge["title_2"]]
    group_fre = df_merge.groupby(["title","title_2"]).count().sort_values("title_ref",ascending=False).reset_index()
    group_fre.columns = ["title","title_2","count","count_2"]
    group_fre = group_fre.drop(columns=["count_2"])
    df_dropdup = df_merge.drop_duplicates(subset=["title","title_2"])
    df_dropdup = df_dropdup.merge(group_fre,on=["title","title_2"],how="left")

    df_ref_merge_id = df_dropdup.merge(df_pub[["title","id"]],on="title",how="left")
    df_ref_merge_id = df_ref_merge_id[df_ref_merge_id.id.notna()]
    df_ref_merge_id = df_ref_merge_id.merge(df_pub[["title","id"]],left_on="title_2",right_on="title",how="inner",suffixes=('','_3'))
    df_ref_merge_id = df_ref_merge_id[["id","id_3","count","year"]]
    df_ref_merge_id = df_ref_merge_id.astype({"id":"int","id_3":"int","count":"int"})
    df_ref_merge_id.columns = ["source","target","weight","year"]
    return df_ref_merge_id


def create_similarity_matrix(list_abstracts:list[str]):
    # Create TF-IDF representation
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(list_abstracts)
    similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return similarity_matrix
def turn_id_to_idx(pub_id:int):
    return pub_id

def turn_idx_to_id(pub_idx:int):
    return pub_idx

def recommend_publications(pub_id:int, similarity_matrix, page_size:int=10, page:int=1):
    # Get similarity scores
    
    similarity_scores = similarity_matrix[pub_id]
    
    # Get indices of publications similar to the target publication
    similar_pub_indices = similarity_scores.argsort()[::-1]
    similar_pub_indices = similar_pub_indices[similar_pub_indices != pub_id]
    similar_pub_indices = similar_pub_indices[(page-1)*page_size:page*page_size]
    similar_pub_indices = [turn_idx_to_id(idx) for idx in similar_pub_indices]
    return similar_pub_indices

def add_rec_to_pub(df_pub:pd.DataFrame):
    df_pub["abstracts_preprocess"] = df_pub["abstracts"].apply(clean_text)
    similiar_matrix = create_similarity_matrix(df_pub["abstracts_preprocess"].to_list())
    df_pub = df_pub.drop(columns="id").reset_index(drop=True).reset_index(names="id")
    df_pub["rec"] = df_pub["id"].apply(lambda x : recommend_publications(x,similiar_matrix))
    df_pub = df_pub.drop(columns="abstracts_preprocess")
    return df_pub

def load_data():
    import pandas as pd
    lat_lon_ciry = pd.DataFrame(columns=["affilcity","lat","lon"])
    df_ref = pd.read_csv("ref_raW.csv")
    df_pub = pd.read_csv("pub_raw.csv")
    
    df_pub = df_pub[(df_pub["abstracts"].notna()) & (df_pub["title"].notna())]
    df_pub["len"] = df_pub["title"].str.len()
    for col in ["affilname","affilcity","affilcountry"]:
        df_pub[col] = df_pub[col].apply(ast.literal_eval)
    return df_ref,df_pub,lat_lon_ciry

def main():
    print("Start")
    df_ref,df_pub,lat_lon_ciry = load_data()
    print("Data loaded")
    df_pub,affiliate_df = proprocess_pub_raw(df_pub)
    affiliate_df = affiliate_df.merge(lat_lon_ciry,on="affilcity",how="left")
    print("Data preprocessed")
    df_pub_preprocess = add_rec_to_pub(df_pub)
    print("Recommendation added")
    df_ref_merge_id = preprocess_df_ref(df_ref,df_pub)
    print("Save data")
    affiliate_df.to_csv("./data_process/affiliation.csv",index=False)
    df_pub_preprocess.to_csv("./data_process/pub_preprocessed.csv",index=False)
    df_ref_merge_id.to_csv("./data_process/ref_preprocessed.csv",index=False)

In [88]:
main()

Start
Data loaded
Data preprocessed
Recommendation added
Save data


# TATA preprocess

In [25]:
dftp = pd.read_csv(r"C:\codepluem\CU\datasci_proj\webscraping\research_csv\06D05M2024Y\paper_info.csv")
dftp.drop(columns=["Unnamed: 0"],inplace=True)

In [26]:
import json
json.loads(str(dftp["affiliation"].iloc[0].replace("\'","\"")))

[{'@_fa': 'true',
  'affilname': 'European Investment Bank',
  'affiliation-city': 'Luxembourg',
  'affiliation-country': 'Luxembourg'}]